In [1]:
import warnings
warnings.filterwarnings('ignore')

import argparse
import time

import numpy as np
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import svds

import torch
from input_data import load_data
from preprocessing import general_train_test_split_edges,biased_train_test_split_edges, bidirectional_train_test_split_edges
from layers import DirectedGCNConvEncoder, DirectedInnerProductDecoder
from layers import SingleLayerDirectedGCNConvEncoder
from models import DirectedGAE

from layers import InnerProductDecoder
from layers import DummyEncoder, DummyPairEncoder

from models import GAE

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--dataset',
                    nargs= '?',
                    default='wisconsin',
                    type=str)
parser.add_argument('--task',
                    nargs= '?',
                    default='task_2',
                    type=str)
parser.add_argument('--model',
                    nargs= '?',
                    default='digae',
                    type=str)
parser.add_argument('--learning_rate',
                    nargs= '?',
                    default=0.01,
                    type=float)
parser.add_argument('--epochs',
                    nargs= '?',
                    default=200,
                    type=int)
parser.add_argument('--hidden',
                    nargs= '?',
                    default=64,
                    type=int)
parser.add_argument('--dimension',
                    nargs= '?',
                    default=32,
                    type=int)
parser.add_argument('--alpha',
                    nargs= '?',
                    default=1.0,
                    type=float)
parser.add_argument('--beta',
                    nargs= '?',
                    default=0.0,
                    type=float)
parser.add_argument('--nb_run',
                    nargs= '?',
                    default=10,
                    type=int)
parser.add_argument('--prop_val',
                    nargs= '?',
                    default=5.0,
                    type=float)
parser.add_argument('--prop_test',
                    nargs= '?',
                    default=10.0,
                    type=float)
parser.add_argument('--verbose',
                    nargs= '?',
                    default=True,
                    type=bool)

parser.add_argument('--self_loops',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--adaptive',
                    nargs= '?',
                    default=False,
                    type=bool)
parser.add_argument('--feature_vector_type',
                    nargs= '?',
                    const=None)
parser.add_argument('--feature_vector_size',
                    nargs= '?',
                    const=None,
                    type=int)
parser.add_argument('--directed',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--logfile',
                    nargs='?',
                    default='logs.json',
                    type=str)
parser.add_argument('--validate',
                    nargs='?',
                    default=False,
                    type=bool)

args,_ = parser.parse_known_args()

In [3]:
def train_single():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)


def test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

def dummy_train_single():
    model.train()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    return float(loss)

def dummy_test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)
    

def train_pair():
    model.train()
    optimizer.zero_grad()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)

def test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def dummy_train_pair():
    model.train()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    return float(loss)

def dummy_test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def identity_features(data, k=None):
    num_nodes = data.x.size(0)
    x = torch.eye(num_nodes)
    u_hat = x.clone()
    v_hat = x.clone()
    return u_hat, v_hat

In [4]:
dummy_run = False
 
if args.verbose:
    print("Loading data...")

device     = 'cpu'
directed   = args.directed

dataset_name    = args.dataset
out_channels    = args.dimension
hidden_channels = args.hidden

compute_features = identity_features

feature_vector_size = args.feature_vector_size

if args.model in ['gae','source_target']:
    train_func = train_single
    test_func  = test_single
elif args.model in ['dummy']:
    train_func = dummy_train_single
    test_func  = dummy_test_single
elif args.model in ['dummy_pair']:
    train_func = dummy_train_pair
    test_func  = dummy_test_pair
else:
    train_func = train_pair
    test_func  = test_pair

val_ratio  = args.prop_val / 100.
test_ratio = args.prop_test / 100.

loaded_data = load_data(dataset_name, directed=directed)

Loading data...


In [5]:
mean_roc  = []
mean_ap   = []
mean_time = []

for i in range(args.nb_run):
    if args.verbose:
        print("Masking test edges...")

    if args.task == 'task_1':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = general_train_test_split_edges(data,
                                                         val_ratio=val_ratio,
                                                         test_ratio=test_ratio,
                                                         directed=directed)
    elif args.task == 'task_2':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = biased_train_test_split_edges(data,
                                                        val_ratio=val_ratio,
                                                        test_ratio=test_ratio,
                                                        directed=directed)
    elif args.task == 'task_3':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = bidirectional_train_test_split_edges(data,
                                                               val_ratio=val_ratio,
                                                               test_ratio=test_ratio,
                                                               directed=directed)     
    else:
        raise ValueError('Undefined task!')

    data = data.to(device)
    train_pos_edge_index = data.train_pos_edge_index.to(device)
    if args.validate is True:
        test_pos_edge_index  = data.val_pos_edge_index.to(device)
        test_neg_edge_index  = data.val_neg_edge_index.to(device)
    else:
        test_pos_edge_index  = data.test_pos_edge_index.to(device)
        test_neg_edge_index  = data.test_neg_edge_index.to(device)

    in_channels          = data.x.shape[1]

    in_channels = data.x.size(0)
    u, v = compute_features(data, in_channels)
    data.u = u
    data.v = v
    data.x = torch.cat([data.u, data.v], dim=1)
    
    u = data.u.to(device)
    v = data.v.to(device)
    print(u.shape, v.shape)

    x = data.x.to(device)

    if args.model == 'digae':
        encoder = DirectedGCNConvEncoder(in_channels, hidden_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                         self_loops=args.self_loops,
                                         adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'digae_single_layer':
        encoder = SingleLayerDirectedGCNConvEncoder(in_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                                    self_loops=args.self_loops,
                                                    adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'dummy':
        encoder = DummyEncoder()
        decoder = InnerProductDecoder()
        model   = GAE(encoder, decoder)
        model   = model.to(device)
        dummy_run = True

    elif args.model == 'dummy_pair':
        encoder = DummyPairEncoder()
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)
        dummy_run =True
    else:
        raise ValueError('Undefined model!')

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=5e-4)

    if args.verbose:
        print("Training...")

    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(args.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        loss = train_func()
        avg_cost = loss
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
        "time=", "{:.5f}".format(time.time() - t))
    mean_time.append(time.time() - t_start)

    if args.verbose:
        print("Testing model...")
    auc, ap = test_func(test_pos_edge_index, test_neg_edge_index)
    roc_score, ap_score = auc, ap
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# if adaptive...
print('=' * 60)
print(args.adaptive)
print(args.model)
print('=' * 60)
    
# Report final results
print("\nTest results for", args.model,
      "model on", args.dataset, "on", args.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Masking test edges...
torch.Size([251, 251]) torch.Size([251, 251])
Training...
Epoch: 0001 train_loss= 1.38710 time= 0.01000
Epoch: 0002 train_loss= 1.38475 time= 0.00700
Epoch: 0003 train_loss= 1.38194 time= 0.00600
Epoch: 0004 train_loss= 1.37775 time= 0.00600
Epoch: 0005 train_loss= 1.37084 time= 0.00600
Epoch: 0006 train_loss= 1.35794 time= 0.00700
Epoch: 0007 train_loss= 1.34209 time= 0.00517
Epoch: 0008 train_loss= 1.31244 time= 0.00200
Epoch: 0009 train_loss= 1.28063 time= 0.00220
Epoch: 0010 train_loss= 1.25067 time= 0.00100
Epoch: 0011 train_loss= 1.20883 time= 0.00612
Epoch: 0012 train_loss= 1.18684 time= 0.00300
Epoch: 0013 train_loss= 1.18511 time= 0.00569
Epoch: 0014 train_loss= 1.17005 time= 0.00742
Epoch: 0015 train_loss= 1.14584 time= 0.00600
Epoch: 0016 train_loss= 1.10337 time= 0.00508
Epoch: 0017 train_loss= 1.14361 time= 0.00200
Epoch: 0018 train_loss= 1.11594 time= 0.00100
Epoch: 0019 train_loss= 1.07733 time= 0.00100
Epoch: 0020 train_loss= 1.06201 time= 0.00300


Epoch: 0190 train_loss= 0.39972 time= 0.00700
Epoch: 0191 train_loss= 0.40573 time= 0.00803
Epoch: 0192 train_loss= 0.39253 time= 0.00699
Epoch: 0193 train_loss= 0.37778 time= 0.00803
Epoch: 0194 train_loss= 0.40610 time= 0.00697
Epoch: 0195 train_loss= 0.36735 time= 0.00693
Epoch: 0196 train_loss= 0.39035 time= 0.00707
Epoch: 0197 train_loss= 0.40442 time= 0.00751
Epoch: 0198 train_loss= 0.37688 time= 0.00749
Epoch: 0199 train_loss= 0.41304 time= 0.00700
Epoch: 0200 train_loss= 0.36301 time= 0.00800
Testing model...
Masking test edges...
torch.Size([251, 251]) torch.Size([251, 251])
Training...
Epoch: 0001 train_loss= 1.38650 time= 0.00701
Epoch: 0002 train_loss= 1.38440 time= 0.00699
Epoch: 0003 train_loss= 1.38121 time= 0.00793
Epoch: 0004 train_loss= 1.37594 time= 0.00707
Epoch: 0005 train_loss= 1.36776 time= 0.00693
Epoch: 0006 train_loss= 1.35466 time= 0.00804
Epoch: 0007 train_loss= 1.33201 time= 0.00735
Epoch: 0008 train_loss= 1.31179 time= 0.00860
Epoch: 0009 train_loss= 1.284

Epoch: 0171 train_loss= 0.43426 time= 0.00792
Epoch: 0172 train_loss= 0.36974 time= 0.00803
Epoch: 0173 train_loss= 0.41824 time= 0.00712
Epoch: 0174 train_loss= 0.37042 time= 0.00883
Epoch: 0175 train_loss= 0.43855 time= 0.00728
Epoch: 0176 train_loss= 0.43123 time= 0.00776
Epoch: 0177 train_loss= 0.39490 time= 0.00797
Epoch: 0178 train_loss= 0.41970 time= 0.00786
Epoch: 0179 train_loss= 0.39873 time= 0.00909
Epoch: 0180 train_loss= 0.40152 time= 0.00790
Epoch: 0181 train_loss= 0.40334 time= 0.00700
Epoch: 0182 train_loss= 0.38740 time= 0.00800
Epoch: 0183 train_loss= 0.39249 time= 0.00557
Epoch: 0184 train_loss= 0.38301 time= 0.00714
Epoch: 0185 train_loss= 0.43506 time= 0.00786
Epoch: 0186 train_loss= 0.38684 time= 0.00700
Epoch: 0187 train_loss= 0.41435 time= 0.00700
Epoch: 0188 train_loss= 0.39462 time= 0.00700
Epoch: 0189 train_loss= 0.42292 time= 0.00800
Epoch: 0190 train_loss= 0.40680 time= 0.00600
Epoch: 0191 train_loss= 0.34721 time= 0.00603
Epoch: 0192 train_loss= 0.38771 ti

Epoch: 0150 train_loss= 0.50538 time= 0.00369
Epoch: 0151 train_loss= 0.46276 time= 0.00641
Epoch: 0152 train_loss= 0.48985 time= 0.01208
Epoch: 0153 train_loss= 0.46813 time= 0.00702
Epoch: 0154 train_loss= 0.49820 time= 0.00710
Epoch: 0155 train_loss= 0.48491 time= 0.00402
Epoch: 0156 train_loss= 0.47336 time= 0.00440
Epoch: 0157 train_loss= 0.47017 time= 0.00503
Epoch: 0158 train_loss= 0.45903 time= 0.00500
Epoch: 0159 train_loss= 0.46616 time= 0.00422
Epoch: 0160 train_loss= 0.48036 time= 0.00350
Epoch: 0161 train_loss= 0.42624 time= 0.00113
Epoch: 0162 train_loss= 0.44038 time= 0.00561
Epoch: 0163 train_loss= 0.46129 time= 0.00098
Epoch: 0164 train_loss= 0.46029 time= 0.00107
Epoch: 0165 train_loss= 0.52627 time= 0.00539
Epoch: 0166 train_loss= 0.48158 time= 0.00355
Epoch: 0167 train_loss= 0.44406 time= 0.01137
Epoch: 0168 train_loss= 0.41469 time= 0.00201
Epoch: 0169 train_loss= 0.47058 time= 0.00605
Epoch: 0170 train_loss= 0.42949 time= 0.00347
Epoch: 0171 train_loss= 0.45371 ti

Epoch: 0141 train_loss= 0.45849 time= 0.00692
Epoch: 0142 train_loss= 0.39612 time= 0.00606
Epoch: 0143 train_loss= 0.41974 time= 0.00640
Epoch: 0144 train_loss= 0.42265 time= 0.00706
Epoch: 0145 train_loss= 0.47079 time= 0.00599
Epoch: 0146 train_loss= 0.38152 time= 0.00596
Epoch: 0147 train_loss= 0.52056 time= 0.00506
Epoch: 0148 train_loss= 0.45223 time= 0.00500
Epoch: 0149 train_loss= 0.45259 time= 0.00499
Epoch: 0150 train_loss= 0.44849 time= 0.00606
Epoch: 0151 train_loss= 0.47693 time= 0.00794
Epoch: 0152 train_loss= 0.50204 time= 0.00646
Epoch: 0153 train_loss= 0.44503 time= 0.00607
Epoch: 0154 train_loss= 0.42469 time= 0.00727
Epoch: 0155 train_loss= 0.44639 time= 0.00900
Epoch: 0156 train_loss= 0.43889 time= 0.00799
Epoch: 0157 train_loss= 0.42462 time= 0.00600
Epoch: 0158 train_loss= 0.47443 time= 0.00600
Epoch: 0159 train_loss= 0.41528 time= 0.00706
Epoch: 0160 train_loss= 0.44940 time= 0.00700
Epoch: 0161 train_loss= 0.45137 time= 0.00799
Epoch: 0162 train_loss= 0.43629 ti

...

Epoch: 0142 train_loss= 0.47737 time= 0.00800
Epoch: 0143 train_loss= 0.49544 time= 0.00993
Epoch: 0144 train_loss= 0.47636 time= 0.00798
Epoch: 0145 train_loss= 0.47347 time= 0.00703
Epoch: 0146 train_loss= 0.42637 time= 0.00599
Epoch: 0147 train_loss= 0.47557 time= 0.00598
Epoch: 0148 train_loss= 0.44913 time= 0.00701
Epoch: 0149 train_loss= 0.46471 time= 0.00759
Epoch: 0150 train_loss= 0.45943 time= 0.00422
Epoch: 0151 train_loss= 0.44719 time= 0.00569
Epoch: 0152 train_loss= 0.42580 time= 0.00843
Epoch: 0153 train_loss= 0.43531 time= 0.00541
Epoch: 0154 train_loss= 0.50793 time= 0.01026
Epoch: 0155 train_loss= 0.40860 time= 0.00414
Epoch: 0156 train_loss= 0.47221 time= 0.01258
Epoch: 0157 train_loss= 0.43165 time= 0.00108
Epoch: 0158 train_loss= 0.43136 time= 0.00650
Epoch: 0159 train_loss= 0.45333 time= 0.00919
Epoch: 0160 train_loss= 0.41635 time= 0.00472
Epoch: 0161 train_loss= 0.42382 time= 0.01051
Epoch: 0162 train_loss= 0.43436 time= 0.00310
Epoch: 0163 train_loss= 0.43646 ti

Epoch: 0132 train_loss= 0.52644 time= 0.00905
Epoch: 0133 train_loss= 0.52991 time= 0.00592
Epoch: 0134 train_loss= 0.50653 time= 0.00704
Epoch: 0135 train_loss= 0.51344 time= 0.00800
Epoch: 0136 train_loss= 0.48212 time= 0.00702
Epoch: 0137 train_loss= 0.54718 time= 0.00698
Epoch: 0138 train_loss= 0.50198 time= 0.00701
Epoch: 0139 train_loss= 0.46075 time= 0.00702
Epoch: 0140 train_loss= 0.51805 time= 0.00694
Epoch: 0141 train_loss= 0.48949 time= 0.00542
Epoch: 0142 train_loss= 0.47607 time= 0.00508
Epoch: 0143 train_loss= 0.49093 time= 0.00605
Epoch: 0144 train_loss= 0.49839 time= 0.00595
Epoch: 0145 train_loss= 0.67677 time= 0.00500
Epoch: 0146 train_loss= 0.56308 time= 0.00507
Epoch: 0147 train_loss= 0.59591 time= 0.00402
Epoch: 0148 train_loss= 0.51824 time= 0.00484
Epoch: 0149 train_loss= 0.53198 time= 0.00547
Epoch: 0150 train_loss= 0.54602 time= 0.00607
Epoch: 0151 train_loss= 0.54201 time= 0.00600
Epoch: 0152 train_loss= 0.49373 time= 0.00600
Epoch: 0153 train_loss= 0.53106 ti

Epoch: 0120 train_loss= 0.57429 time= 0.00626
Epoch: 0121 train_loss= 0.58449 time= 0.00761
Epoch: 0122 train_loss= 0.54426 time= 0.00691
Epoch: 0123 train_loss= 0.54766 time= 0.00805
Epoch: 0124 train_loss= 0.61950 time= 0.00802
Epoch: 0125 train_loss= 0.54052 time= 0.00691
Epoch: 0126 train_loss= 0.51736 time= 0.00707
Epoch: 0127 train_loss= 0.57030 time= 0.00700
Epoch: 0128 train_loss= 0.48716 time= 0.00601
Epoch: 0129 train_loss= 0.51831 time= 0.00481
Epoch: 0130 train_loss= 0.49405 time= 0.00516
Epoch: 0131 train_loss= 0.53488 time= 0.00609
Epoch: 0132 train_loss= 0.51753 time= 0.00456
Epoch: 0133 train_loss= 0.51900 time= 0.00525
Epoch: 0134 train_loss= 0.50478 time= 0.00600
Epoch: 0135 train_loss= 0.50280 time= 0.00442
Epoch: 0136 train_loss= 0.47269 time= 0.00523
Epoch: 0137 train_loss= 0.49414 time= 0.00604
Epoch: 0138 train_loss= 0.49845 time= 0.00585
Epoch: 0139 train_loss= 0.50990 time= 0.00799
Epoch: 0140 train_loss= 0.51489 time= 0.00702
Epoch: 0141 train_loss= 0.47123 ti

Epoch: 0111 train_loss= 0.49962 time= 0.00600
Epoch: 0112 train_loss= 0.54327 time= 0.00586
Epoch: 0113 train_loss= 0.51217 time= 0.00493
Epoch: 0114 train_loss= 0.56678 time= 0.00734
Epoch: 0115 train_loss= 0.45551 time= 0.00694
Epoch: 0116 train_loss= 0.46717 time= 0.00605
Epoch: 0117 train_loss= 0.45654 time= 0.00602
Epoch: 0118 train_loss= 0.48973 time= 0.00899
Epoch: 0119 train_loss= 0.48456 time= 0.00700
Epoch: 0120 train_loss= 0.53695 time= 0.00700
Epoch: 0121 train_loss= 0.50335 time= 0.00700
Epoch: 0122 train_loss= 0.46171 time= 0.00698
Epoch: 0123 train_loss= 0.46318 time= 0.00602
Epoch: 0124 train_loss= 0.45859 time= 0.00700
Epoch: 0125 train_loss= 0.46613 time= 0.00301
Epoch: 0126 train_loss= 0.43671 time= 0.00108
Epoch: 0127 train_loss= 0.47709 time= 0.00111
Epoch: 0128 train_loss= 0.46474 time= 0.00312
Epoch: 0129 train_loss= 0.45583 time= 0.00107
Epoch: 0130 train_loss= 0.44814 time= 0.00107
Epoch: 0131 train_loss= 0.46165 time= 0.00299
Epoch: 0132 train_loss= 0.44668 ti

Epoch: 0102 train_loss= 0.49165 time= 0.00706
Epoch: 0103 train_loss= 0.49615 time= 0.00501
Epoch: 0104 train_loss= 0.58678 time= 0.00590
Epoch: 0105 train_loss= 0.56759 time= 0.00706
Epoch: 0106 train_loss= 0.57545 time= 0.00694
Epoch: 0107 train_loss= 0.50752 time= 0.00707
Epoch: 0108 train_loss= 0.48845 time= 0.00694
Epoch: 0109 train_loss= 0.54705 time= 0.00706
Epoch: 0110 train_loss= 0.54963 time= 0.00700
Epoch: 0111 train_loss= 0.54122 time= 0.00701
Epoch: 0112 train_loss= 0.52176 time= 0.00599
Epoch: 0113 train_loss= 0.51806 time= 0.00692
Epoch: 0114 train_loss= 0.57421 time= 0.00607
Epoch: 0115 train_loss= 0.52592 time= 0.00641
Epoch: 0116 train_loss= 0.50199 time= 0.00757
Epoch: 0117 train_loss= 0.49379 time= 0.00700
Epoch: 0118 train_loss= 0.49751 time= 0.00798
Epoch: 0119 train_loss= 0.51875 time= 0.00702
Epoch: 0120 train_loss= 0.49067 time= 0.00700
Epoch: 0121 train_loss= 0.47866 time= 0.00700
Epoch: 0122 train_loss= 0.48556 time= 0.00795
Epoch: 0123 train_loss= 0.46531 ti